In [1]:
from gensim.models import Word2Vec, KeyedVectors
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.utils import class_weight
import torch
from gensim.scripts.glove2word2vec import glove2word2vec
import pandas as pd
import numpy as np
import random
import re
import os,sys
import gensim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.optim.lr_scheduler import ReduceLROnPlateau as lr_scheduler
from torch.optim.lr_scheduler import StepLR
import num2words
from num2words import num2words
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
unknown_tag = "<unknown_tag>"
number_tag = "<number_tag>"
curr_batch_len = 8


In [2]:
my_num_list = []
for i in range(100):
    my_num_list.append(num2words(i))
other_number = ['hundred', 'thousand', 'million', 'billion', 'trillion', 'quadrillion', 'quintillion', 'sextillion', 'septillion', 'octillion', 'nonillion', 'decillion']
for i in other_number:
    my_num_list.append(i)

   
    

In [3]:
def process_input(filename):

    f=open(os.path.join(sys.path[0],filename),"r")
    lines=f.read().splitlines()
    curr_sentence =[]
    curr_word = []
    get_all_words = []
    for x in lines:
        y = x.split()

        if len(y)==3:
            curr_sentence.append(y)
            curr_word.append(y[1])
        else:
          get_all_words.append(curr_word)
          curr_word = []
    get_all_words.append(curr_word)
        
    f.close()


    return curr_sentence,get_all_words
    

In [4]:
pre_train,pre_train_words = process_input("train")
pre_dev,pre_dev_words = process_input("dev")


In [6]:
def process_test_input(filename):
    f=open(os.path.join(sys.path[0],filename),"r")
    lines=f.read().splitlines()
    curr_sentence =[]
    curr_word = []
    get_all_words = []
    for x in lines:
        y = x.split()
        if len(y)==2:
            curr_sentence.append(y)
            curr_word.append(y[1])
        else:
          get_all_words.append(curr_word)
          curr_word = []
        
    get_all_words.append(curr_word)
    f.close()


    return curr_sentence,get_all_words
    

In [7]:
pre_test,pre_test_words = process_test_input("test")

In [8]:
def chk_num_tag(given_string):
    try:
        
        f=filter(str.isalnum,given_string)
        given_string="".join(f)
        
        if given_string.lower() in my_num_list:
            
            return True
        
        elif given_string.isdecimal():
            return True
        
        float(given_string)
        return True
    except ValueError:
            return False
        
    

In [9]:
curr_indx = 0
all_ner_tags = []
my_vocab = {}
wordidx_dict = {}
tagidx_dict = {}

for curr_line in pre_train:
    my_word = curr_line[1]
    my_tag = curr_line[2]
    if chk_num_tag(my_word):
        my_word = number_tag
    if my_word in my_vocab:
        my_vocab[my_word] += 1
    else:
        my_vocab[my_word] = 1
    if my_tag not in tagidx_dict:
        tagidx_dict[my_tag] = curr_indx
        
        curr_indx += 1
    all_ner_tags.append(tagidx_dict[my_tag])

my_vocab["<unknown_tag>"] = 0


curr_indx = 0
for my_word in my_vocab:
    if my_word not in wordidx_dict:
        wordidx_dict[my_word] = curr_indx
        
        curr_indx += 1


In [12]:
print(len(my_vocab))
print(tagidx_dict)

20178
{'B-ORG': 0, 'O': 1, 'B-MISC': 2, 'B-PER': 3, 'I-PER': 4, 'B-LOC': 5, 'I-ORG': 6, 'I-MISC': 7, 'I-LOC': 8}


In [13]:
def make_x_y_blstm(pre_data):
    blstm_word_idx = []
    blstm_tag_idx = []
    list_of_sen = []
    list_of_sen.append(pre_data[0])
    for i in range(1,len(pre_data)):
        if pre_data[i][0] != '1':
            list_of_sen.append(pre_data[i])
        else:
            words = np.array([])
            tags = np.array([])
            my_word =''
            for curr_indx in range(len(list_of_sen)):
                my_word = list_of_sen[curr_indx][1]
                if my_word not in wordidx_dict:
         
                    if chk_num_tag(list_of_sen[curr_indx][1]):
                        my_word  = number_tag

                    else:
                        my_word  = unknown_tag
       
                words= np.append(words,wordidx_dict[my_word])
                tags= np.append(tags,tagidx_dict[list_of_sen[curr_indx][2]])            
            
            blstm_word_idx.append(words)
            blstm_tag_idx.append(tags)

            list_of_sen = []
            list_of_sen.append(pre_data[i])
    
    
    blstm_word_idx.append(np.array([257.]))
    blstm_tag_idx.append(np.array([1]))

    return np.array(blstm_word_idx), np.array(blstm_tag_idx)

In [14]:
train_blstm_x, train_blstm_y = make_x_y_blstm(pre_train)
dev_blstm_x, dev_blstm_y = make_x_y_blstm(pre_dev)

/var/folders/17/jys7jgwn3x37cltk9w8j0l080000gq/T/ipykernel_59682/258108467.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(blstm_word_idx), np.array(blstm_tag_idx)


In [22]:
print(train_blstm_x[1])

[ 9. 10.]


In [23]:
class PadSequence:
    def __call__(self, batch):
      sorted_batch = sorted(batch, key=lambda x: x[0].shape[0], reverse=True)
      sequences = [torch.LongTensor(x[0]) for x in sorted_batch]
      sequences_padded = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True,padding_value=0)
      lengths = torch.LongTensor([len(x) for x in sequences])
      labels = [torch.LongTensor(x[1]) for x in sorted_batch]
      labels_padded = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True,padding_value=-1)
      return torch.LongTensor(sequences_padded),  torch.LongTensor(labels_padded),lengths


      

In [24]:
task1_train_dataset = list(zip(train_blstm_x,train_blstm_y))
task1_dev_dataset = list(zip(dev_blstm_x,dev_blstm_y))

task1_train_loader = DataLoader(task1_train_dataset, batch_size = curr_batch_len, drop_last = True, shuffle = True,collate_fn=PadSequence())
task1_dev_loader = DataLoader(task1_dev_dataset, batch_size = curr_batch_len, drop_last = True, shuffle = True,collate_fn=PadSequence())


In [28]:
print((task1_train_loader[0]))

TypeError: 'DataLoader' object is not subscriptable

In [52]:
ip_dim = len(wordidx_dict)
op_dim = len(tagidx_dict)
em_dim = 100
hidden_dim = 256
fc_dim = 128


In [53]:
class task1_simple_BLSTM(torch.nn.Module):
    def __init__(self, ip_dim, em_dim, hidden_dim, fc_dim, op_dim):
        super(task1_simple_BLSTM, self).__init__()
        self.embed_layer = torch.nn.Embedding(num_embeddings = ip_dim, embedding_dim = em_dim)
        self.blstm_layer = torch.nn.LSTM(input_size = em_dim, hidden_size = hidden_dim, num_layers = 1, bidirectional = True, batch_first = True, dropout = 0.33)
        self.linear_layer = torch.nn.Linear(hidden_dim*2, fc_dim)
        self.elu_layer = torch.nn.ELU()
        self.classifier_layer = torch.nn.Linear(fc_dim, op_dim)
    def forward(self, x,x_len):
        embed_layer_out = self.embed_layer(x)
        blstm_packed = pack_padded_sequence(embed_layer_out,x_len,batch_first=True,enforce_sorted=True)
        blstm_layer_out, _ = self.blstm_layer(blstm_packed)
        blstm_layer_out, _ = pad_packed_sequence(blstm_layer_out, batch_first=True)
        classifier_layer_out = self.elu_layer(self.linear_layer(blstm_layer_out))
        target_output = self.classifier_layer(classifier_layer_out)
        return target_output

    
    def wt_initalize(self):
        for name, param in self.named_parameters():
            torch.nn.init.normal_(param.data, mean=0, std=0.1)

    

In [ ]:
del model, optimizer, loss_fn

In [55]:
model = task1_simple_BLSTM(ip_dim, em_dim, hidden_dim, fc_dim, op_dim).to(device)
model.wt_initalize()
print(model)

task1_simple_BLSTM(
  (embed_layer): Embedding(20178, 100)
  (blstm_layer): LSTM(100, 256, batch_first=True, dropout=0.33, bidirectional=True)
  (linear_layer): Linear(in_features=512, out_features=128, bias=True)
  (elu_layer): ELU(alpha=1.0)
  (classifier_layer): Linear(in_features=128, out_features=9, bias=True)
)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.33 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [56]:
tag_weightage = torch.FloatTensor(class_weight.compute_class_weight('balanced', np.unique(all_ner_tags), all_ner_tags))
loss_fn = torch.nn.CrossEntropyLoss(weight = tag_weightage, ignore_index = -1).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.017,momentum = 0.9)
scheduler = StepLR(optimizer, step_size = 25) 

tensor([ 3.5959,  0.1333,  6.6113,  3.4439,  5.0198,  3.1834,  6.1365, 19.6794,
        19.6453])


In [57]:
def find_epoch_accuracy(pred,targ):
  pred = pred.argmax(dim=1,keepdim=True)
  find_targ_val = (targ!=-1).nonzero()
  correct = pred[find_targ_val].squeeze(1).eq(targ[find_targ_val])
  return correct.sum() / torch.FloatTensor([targ[find_targ_val].shape[0]]).to(device)

In [58]:
epochs = 20
devset_min_loss = np.inf
devset_max_accuracy = 0
curr_train_loader = task1_train_loader
curr_dev_loader = task1_dev_loader


for epoch in range(epochs):
    model.train()
    epoch_train_accuracy = 0.0
    epoch_dev_accuracy = 0.0
    trainset_loss = 0.0
    devset_loss = 0.0
    
    for ip_sen, targ_ner,x_len in curr_train_loader:
        ip_sen, targ_ner,x_len = ip_sen.to(device), targ_ner.to(device),x_len
        optimizer.zero_grad()
        output = model(ip_sen,x_len)
        output = output.view(-1, output.shape[-1])
        targ_ner = targ_ner.view(-1)
        loss = loss_fn(output, targ_ner)
        loss.backward()
        optimizer.step()
        epoch_train_accuracy += float(find_epoch_accuracy(output,targ_ner).item()) 
        trainset_loss += loss.item()
    
    model.eval()

    for ip_sen, targ_ner,x_len in curr_dev_loader:
        ip_sen, targ_ner = ip_sen.to(device), targ_ner.to(device)
        ip_sen, targ_ner = ip_sen, targ_ner
        output = model(ip_sen,x_len)
        output = output.view(-1, output.shape[-1])
        targ_ner = targ_ner.view(-1)
        loss = loss_fn(output, targ_ner)
        epoch_dev_accuracy += float(find_epoch_accuracy(output,targ_ner).item()) 
        devset_loss += loss.item()
    
    epoch_train_accuracy = (epoch_train_accuracy * curr_batch_len)/len(curr_train_loader.dataset)
    epoch_dev_accuracy = (epoch_dev_accuracy * curr_batch_len)/len(curr_dev_loader.dataset)
    trainset_loss = trainset_loss/len(curr_train_loader.dataset)
    devset_loss = devset_loss/len(curr_dev_loader.dataset)
    
    
    print('Epoch:',epoch+1,'\nTraining Loss:', trainset_loss,' Devset Loss:',devset_loss)
    print('Training Accuracy:',epoch_train_accuracy,' Devset Accuracy:',epoch_dev_accuracy)
    print('\n')
    if epoch_dev_accuracy >= devset_max_accuracy:
        print('Devset Accuracy increased hence saving model as checkpoint')
        print('\n')
        print("Previous Best:",devset_max_accuracy," Current Best:",epoch_dev_accuracy)
        print('\n')
        torch.save(model.state_dict(), os.path.join(sys.path[0],'/content/drive/MyDrive/Colab Notebooks/data/blstm1.pt'))
        devset_max_accuracy = epoch_dev_accuracy
    scheduler.step(devset_max_accuracy)

Epoch: 1 
Training Loss: 0.16193246425725352  Devset Loss: 0.10054690041727092
Training Accuracy: 0.575452885150466  Devset Accuracy: 0.7760861672491127


Devset Accuracy increased hence saving model as checkpoint


Previous Best: 0  Current Best: 0.7760861672491127




/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch: 2 
Training Loss: 0.06424895281973479  Devset Loss: 0.06924725811610367
Training Accuracy: 0.8029010481518472  Devset Accuracy: 0.828526852107502


Devset Accuracy increased hence saving model as checkpoint


Previous Best: 0.7760861672491127  Current Best: 0.828526852107502


Epoch: 3 
Training Loss: 0.030167968466215753  Devset Loss: 0.07920056530846786
Training Accuracy: 0.8887509689119791  Devset Accuracy: 0.8901260713587176


Devset Accuracy increased hence saving model as checkpoint


Previous Best: 0.828526852107502  Current Best: 0.8901260713587176


Epoch: 4 
Training Loss: 0.016253252220303098  Devset Loss: 0.07993777479844213
Training Accuracy: 0.9328758752058893  Devset Accuracy: 0.9193809840009175


Devset Accuracy increased hence saving model as checkpoint


Previous Best: 0.8901260713587176  Current Best: 0.9193809840009175


Epoch: 5 
Training Loss: 0.010567443027575113  Devset Loss: 0.0786224362824383
Training Accuracy: 0.9548599400402284  Devset Accuracy: 0.923

In [59]:
del model, optimizer, loss_fn

In [61]:
def make_output_file(x, y, pre_data,loaded_model,op_filename, test_file = False):
    loaded_model.eval()
    line_num = 0
    me = 0
    
    with torch.no_grad():
        with open(op_filename, "w") as fp:
            for i in range(len(x)):
              curr_indx = 1
              x_len = np.array([len(x[i])])
              ip = torch.LongTensor(x[i]).to(device)
              ip = torch.unsqueeze(ip, 0)
              op = loaded_model(ip,x_len)
              op = op.view(-1, op.shape[-1])
              _, pred = torch.max(op, 1)

              if not test_file:
                targ_ner = y[i]
                for j in range(len(targ_ner)):
                    
                    pred_tag = int(pred[j])
                    targ_tag = int(targ_ner[j])
                    
                    z = pre_data[line_num][1]
                    
                    for key,value in tagidx_dict.items():
                      if value == targ_tag:
                        gold = key
                      if value == pred_tag:
                        my_word = key
                    
                    my_sen_pred = str(curr_indx)+' '+z+' '+gold+' '+my_word+'\n'
                    fp.write(my_sen_pred)
                    curr_indx += 1
                    line_num += 1
                fp.write('\n')
              else:
                for j in range(len(pred)):
                  pred_tag = int(pred[j])
                  z = pre_data[line_num][1]
                  for key,value in tagidx_dict.items():
                      if value == pred_tag:
                        my_word = key
                  my_sen_pred = str(curr_indx)+' '+z+' '+my_word+'\n'
                  fp.write(my_sen_pred)
                  curr_indx += 1
                  line_num += 1
                fp.write('\n')


In [62]:
del load_model

NameError: ignored

In [63]:
load_model = task1_simple_BLSTM(ip_dim, em_dim, hidden_dim, fc_dim, op_dim).to(device)
load_model.load_state_dict(torch.load(os.path.join(sys.path[0],'/content/drive/MyDrive/Colab Notebooks/data/blstm1.pt')))
make_output_file(dev_blstm_x, dev_blstm_y, pre_dev, load_model, os.path.join(sys.path[0],'/content/drive/MyDrive/Colab Notebooks/data/dev1.out'))


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.33 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


False


In [65]:
def make_test_x_y_blstm(pre_data):
    blstm_word_idx = []
    #blstm_tag_idx = []
    list_of_sen = []
    list_of_sen.append(pre_data[0])
    for i in range(1,len(pre_data)):
        if pre_data[i][0] != '1':
            list_of_sen.append(pre_data[i])
        else:
            words = np.array([])
            #tags = np.array([])
            my_word =''
            for curr_indx in range(len(list_of_sen)):
                my_word = list_of_sen[curr_indx][1]
                if my_word not in wordidx_dict:
         
                    if chk_num_tag(list_of_sen[curr_indx][1]):
                        my_word  = number_tag

                    else:
                        my_word  = unknown_tag
       
                words= np.append(words,wordidx_dict[my_word])
                #tags= np.append(tags,tagidx_dict[list_of_sen[curr_indx][2]])            
            
            blstm_word_idx.append(words)
            #blstm_tag_idx.append(tags)

            list_of_sen = []
            list_of_sen.append(pre_data[i])
    
    
    blstm_word_idx.append(np.array([257.]))
    #blstm_tag_idx.append(np.array([1]))

    return np.array(blstm_word_idx)

In [66]:
test_blstm_x = make_test_x_y_blstm(pre_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [68]:
load_model = task1_simple_BLSTM(ip_dim, em_dim, hidden_dim, fc_dim, op_dim).to(device)
load_model.load_state_dict(torch.load(os.path.join(sys.path[0],'/content/drive/MyDrive/Colab Notebooks/data/blstm1.pt')))
make_output_file(test_blstm_x, [], pre_test, load_model, os.path.join(sys.path[0],'/content/drive/MyDrive/Colab Notebooks/data/test1.out'),True)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.33 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


True


In [31]:
all_words = pre_train_words + pre_dev_words + pre_test_words
glove_w2v_file = 'glove.txt.word2vec'
glove2word2vec("glove.6B.100d.gz", glove_w2v_file)
glove_vec = KeyedVectors.load_word2vec_format(glove_w2v_file)

In [32]:
print(len(all_words))

22137


In [33]:
all_wordidx_dict = {}
curr_indx = 0

for t2_curr_word in all_words:
    for my_word in t2_curr_word:
        if my_word not in all_wordidx_dict:
            all_wordidx_dict[my_word] = curr_indx
            curr_indx += 1
    all_wordidx_dict[number_tag] = curr_indx

In [36]:
embed_weight_mat = np.zeros((len(all_wordidx_dict), 100), dtype = float)
glove_embed_dict = {}


In [37]:
def glove_embed_weight(embed_weight_mat, model):
    global all_wordidx_dict, glove_embed_dict
    for my_word, curr_indx in all_wordidx_dict.items():
        my_word = my_word.lower()
        
        if my_word in glove_embed_dict:
            embed_weight_mat[curr_indx] = glove_embed_dict[my_word]
        else:
          if my_word in model.vocab:
            embed_weight_mat[curr_indx] = model[my_word]
          else:
            rand_embed = np.random.normal(scale = 0.6, size = (100,))
            embed_weight_mat[curr_indx] = rand_embed
            glove_embed_dict[my_word] = rand_embed

            
    
    return embed_weight_mat

In [38]:
embed_weight_mat = glove_embed_weight(embed_weight_mat, glove_vec)


In [41]:
print(len(embed_weight_mat[0]))

100


In [42]:
def glove_make_x_y_blstm(pre_data,test = False):
    blstm_word_idx = []
    blstm_tag_idx = []
    list_of_sen = []
    list_of_sen.append(pre_data[0])
    for i in range(1,len(pre_data)):
        if pre_data[i][0] != '1':
            list_of_sen.append(pre_data[i])
        else:
            words = np.array([])
            tags = np.array([])
            my_word =''
            for curr_indx in range(len(list_of_sen)):
                my_word = list_of_sen[curr_indx][1]
                if chk_num_tag(my_word):

                  my_word  = number_tag              
       
                words= np.append(words,all_wordidx_dict[my_word])
                if not test:
                    tags= np.append(tags,tagidx_dict[list_of_sen[curr_indx][2]])            
            
            blstm_word_idx.append(words)
            if not test:
                blstm_tag_idx.append(tags)

            list_of_sen = []
            list_of_sen.append(pre_data[i])
    
    
    blstm_word_idx.append(np.array([257.]))
    if not test:
        blstm_tag_idx.append(np.array([1]))

    return np.array(blstm_word_idx), np.array(blstm_tag_idx)

In [43]:
glove_train_blstm_x, glove_train_blstm_y = glove_make_x_y_blstm(pre_train)
glove_dev_blstm_x, glove_dev_blstm_y = glove_make_x_y_blstm(pre_dev)
glove_test_blstm_x,_ = glove_make_x_y_blstm(pre_test,True)


/var/folders/17/jys7jgwn3x37cltk9w8j0l080000gq/T/ipykernel_59682/3584130673.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(blstm_word_idx), np.array(blstm_tag_idx)


In [44]:
print(glove_train_blstm_x[0])

[0. 1. 2. 3. 4. 5. 6. 7. 8.]


In [45]:
glove_train_dataset = list(zip(glove_train_blstm_x,glove_train_blstm_y))
glove_dev_dataset = list(zip(glove_dev_blstm_x,glove_dev_blstm_y))

glove_train_loader = DataLoader(glove_train_dataset, batch_size = curr_batch_len, drop_last = True, shuffle = True,collate_fn=PadSequence())
glove_dev_loader = DataLoader(glove_dev_dataset, batch_size = curr_batch_len, drop_last = True, shuffle = True,collate_fn=PadSequence())


In [47]:
ip_dim = len(all_wordidx_dict)
em_dim = 100
hidden_dim = 256
fc_dim = 128
op_dim = len(tagidx_dict)
tag_weightage = class_weight.compute_class_weight('balanced', np.unique(all_ner_tags), all_ner_tags)

In [76]:
del model, optimizer

In [77]:
def find_embed_layer(ip_dim, em_dim, embed_weight_mat):
    embed_weight_mat = torch.FloatTensor(embed_weight_mat).to(device)
    embed_layer = torch.nn.Embedding(num_embeddings = ip_dim, embedding_dim = em_dim)
    embed_layer.load_state_dict({'weight': embed_weight_mat})
    return embed_layer

In [78]:
class task2_glove_BLSTM(torch.nn.Module):
    def __init__(self, ip_dim, em_dim, hidden_dim, fc_dim, op_dim, embed_weight_mat):
        super(task2_glove_BLSTM, self).__init__()
        self.embed_layer = find_embed_layer(ip_dim, em_dim, embed_weight_mat)
        self.blstm_layer = torch.nn.LSTM(input_size = em_dim, hidden_size = hidden_dim, num_layers = 1, bidirectional = True, batch_first = True, dropout = 0.33)
        self.linear_layer = torch.nn.Linear(hidden_dim * 2, fc_dim)
        self.elu_layer = torch.nn.ELU()
        self.classifier_layer = torch.nn.Linear(fc_dim, op_dim)
    
    def forward(self, x,x_len):
        print('x:',x.shape)
        emb = self.embed_layer(x)
        print('emb:',emb.shape)
        blstm_packed = pack_padded_sequence(emb, x_len, batch_first = True, enforce_sorted = True)
        print('hd:',len(blstm_packed))
        blstm_layer_out, _ = self.blstm_layer(blstm_packed)
        blstm_layer_out, _ = pad_packed_sequence(blstm_layer_out, batch_first = True)
        classifier_layer_out = self.elu_layer(self.linear_layer(blstm_layer_out))
        target_output = self.classifier_layer(classifier_layer_out)
        return target_output
    

In [79]:
model = task2_glove_BLSTM(ip_dim, em_dim, hidden_dim, fc_dim, op_dim, embed_weight_mat).to(device)
print(model)

task2_glove_BLSTM(
  (embed_layer): Embedding(30291, 100)
  (blstm_layer): LSTM(100, 256, batch_first=True, dropout=0.33, bidirectional=True)
  (linear_layer): Linear(in_features=512, out_features=128, bias=True)
  (elu_layer): ELU(alpha=1.0)
  (classifier_layer): Linear(in_features=128, out_features=9, bias=True)
)


In [80]:
tag_weightage = torch.FloatTensor(tag_weightage)
loss_fn = torch.nn.CrossEntropyLoss(weight = tag_weightage, ignore_index = -1).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.017, momentum = 0.9)
scheduler = StepLR(optimizer, step_size = 25) 


In [85]:
epochs = 1
devset_min_loss = np.inf
devset_max_accuracy = 0
curr_train_loader = glove_train_loader
curr_dev_loader = glove_dev_loader


for epoch in range(epochs):
    model.train()
    epoch_train_accuracy = 0.0
    epoch_dev_accuracy = 0.0
    trainset_loss = 0.0
    devset_loss = 0.0
    i = 0
    for ip_sen, targ_ner,x_len in curr_train_loader:
        print('ip:',ip_sen[0].shape,x_len)
        if i==1:
            break
        i+=1
        ip_sen, targ_ner,x_len = ip_sen.to(device), targ_ner.to(device),x_len
        optimizer.zero_grad()
        output = model(ip_sen,x_len)
        print('op:',output.shape)
        print('t:',output.view(-1,output.shape[-1]).shape)
        output = output.view(-1, output.shape[-1])
        print('finalt:',output[0])
        targ_ner = targ_ner.view(-1)
        print('tar:',targ_ner)
        loss = loss_fn(output, targ_ner)
        loss.backward()
        optimizer.step()
        epoch_train_accuracy += float(find_epoch_accuracy(output,targ_ner).item()) 
        trainset_loss += loss.item()
    
    model.eval()

    for ip_sen, targ_ner,x_len in curr_dev_loader:
        if i==1:
            break
        ip_sen, targ_ner = ip_sen.to(device), targ_ner.to(device)
        ip_sen, targ_ner = ip_sen, targ_ner
        output = model(ip_sen,x_len)
        output = output.view(-1, output.shape[-1])
        targ_ner = targ_ner.view(-1)
        loss = loss_fn(output, targ_ner)
        epoch_dev_accuracy += float(find_epoch_accuracy(output,targ_ner).item()) 
        devset_loss += loss.item()
    
    epoch_train_accuracy = (epoch_train_accuracy * curr_batch_len)/len(curr_train_loader.dataset)
    epoch_dev_accuracy = (epoch_dev_accuracy * curr_batch_len)/len(curr_dev_loader.dataset)
    trainset_loss = trainset_loss/len(curr_train_loader.dataset)
    devset_loss = devset_loss/len(curr_dev_loader.dataset)
    
    
    print('Epoch:',epoch+1,'\nTraining Loss:', trainset_loss,' Devset Loss:',devset_loss)
    print('Training Accuracy:',epoch_train_accuracy,' Devset Accuracy:',epoch_dev_accuracy)
    print('\n')
    if epoch_dev_accuracy >= devset_max_accuracy:
        print('Devset Accuracy increased hence saving model as checkpoint')
        print('\n')
        print("Previous Best:",devset_max_accuracy," Current Best:",epoch_dev_accuracy)
        print('\n')
        torch.save(model.state_dict(), os.path.join(sys.path[0],'/content/drive/MyDrive/Colab Notebooks/data/blstm2.pt'))
        devset_max_accuracy = epoch_dev_accuracy
    scheduler.step(devset_max_accuracy)

ip: torch.Size([40]) tensor([40, 35, 33, 30, 22, 12,  5,  4])
x: torch.Size([8, 40])
emb: torch.Size([8, 40, 100])
hd: 4
op: torch.Size([8, 40, 9])
t: torch.Size([320, 9])
finalt: tensor([ 0.0370, -0.0612,  0.0275,  0.0570,  0.0579,  0.0130, -0.0389, -0.0656,
         0.0131], grad_fn=<SelectBackward>)
tar: tensor([ 2,  1,  3,  4,  1,  1,  1,  1,  1,  3,  4,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  5,  1,  1,  1,  1,  1,  1,  3,  1,  1,  1,  1,  1,  5,  1,  0,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  5,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  5,  1,  1,  1,  2,  1,  1,  5,  8,  1,  1,  1,  1,  1,  2,  1,
         1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  5,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1,

NameError: name 'find_epoch_accuracy' is not defined

In [126]:
del load_model

In [127]:
load_model = task2_glove_BLSTM(ip_dim, em_dim, hidden_dim, fc_dim, op_dim,embed_weight_mat).to(device)
load_model.load_state_dict(torch.load(os.path.join(sys.path[0],'/content/drive/MyDrive/Colab Notebooks/data/blstm2.pt') ))
make_output_file(glove_dev_blstm_x, glove_dev_blstm_y, pre_dev, load_model, os.path.join(sys.path[0],'/content/drive/MyDrive/Colab Notebooks/data/dev2.out'))


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.33 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


False


In [128]:
del load_model

In [129]:
load_model = task2_glove_BLSTM(ip_dim, em_dim, hidden_dim, fc_dim, op_dim,embed_weight_mat).to(device)
load_model.load_state_dict(torch.load(os.path.join(sys.path[0],'/content/drive/MyDrive/Colab Notebooks/data/blstm2.pt')))
make_output_file(glove_test_blstm_x, [], pre_test, load_model, os.path.join(sys.path[0],'/content/drive/MyDrive/Colab Notebooks/data/test2.out'),True)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.33 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


True
